In [1]:
import syft as sy
import torch as th
from syft.lib.python.collections import OrderedDict
import collections

In [2]:
alice = sy.VirtualMachine()
alice_client = alice.get_root_client()

[2021-03-25T12:32:07.142311+0800][CRITICAL][logger]][2012] Skipping torchvision.torchvision.transforms.functional.adjust_sharpness not supported in 0.8.1
[2021-03-25T12:32:07.142690+0800][CRITICAL][logger]][2012] Skipping torchvision.torchvision.transforms.functional.autocontrast not supported in 0.8.1
[2021-03-25T12:32:07.142919+0800][CRITICAL][logger]][2012] Skipping torchvision.torchvision.transforms.functional.equalize not supported in 0.8.1
[2021-03-25T12:32:07.143149+0800][CRITICAL][logger]][2012] Skipping torchvision.torchvision.transforms.functional.invert not supported in 0.8.1
[2021-03-25T12:32:07.143352+0800][CRITICAL][logger]][2012] Skipping torchvision.torchvision.transforms.functional.posterize not supported in 0.8.1
[2021-03-25T12:32:07.143586+0800][CRITICAL][logger]][2012] Skipping torchvision.torchvision.transforms.functional.solarize not supported in 0.8.1


In [3]:
# Linear
fc = th.nn.Linear(4,2)

# send
fc_ptr = fc.send(alice_client)
print(f"----fc_ptr----\n{fc_ptr}\n")

# remote call
res_ptr = fc_ptr(th.rand([1,4]))
print(f"----res_ptr----\n{res_ptr}\n")
print(f"----res_ptr.get()----\n{res_ptr.get()}\n")

# remote update state dict
sd2 = OrderedDict(th.nn.Linear(4,2).state_dict())
sd2_ptr = sd2.send(alice_client)
fc_ptr.load_state_dict(sd2_ptr)

# get
print(f"----fc_ptr.get().state_dict()----\n{fc_ptr.get().state_dict()}\n")
print(f"----sd2----\n{sd2}\n")

----fc_ptr----

----res_ptr----

----res_ptr.get()----
tensor([[-0.2959, -0.1966]], requires_grad=True)

----fc_ptr.get().state_dict()----
OrderedDict([('weight', tensor([[-0.3971,  0.1255,  0.4036, -0.4374],
        [-0.4251, -0.1996,  0.0949,  0.4059]])), ('bias', tensor([ 0.3382, -0.2079]))])

----sd2----
OrderedDict([('weight', tensor([[-0.3971,  0.1255,  0.4036, -0.4374],
        [-0.4251, -0.1996,  0.0949,  0.4059]])), ('bias', tensor([ 0.3382, -0.2079]))])



/mnt/d/github/PySyft/src/syft/lib/torch/uppercase_tensor.py:30: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  grad = getattr(obj, "grad", None)


In [4]:
# ReLU
relu = th.nn.ReLU(inplace=True)

# send
relu_ptr = relu.send(alice_client)
print(f"----relu_ptr----\n{relu_ptr}\n")

# remote call
res_ptr = relu_ptr(th.rand([1,4]))
print(f"----res_ptr----\n{res_ptr}\n")
print(f"----res_ptr.get()----\n{res_ptr.get()}\n")

# get
print(f"----relu_ptr.get()----\n{relu_ptr.get()}\n")

[2021-03-25T12:32:07.222155+0800][CRITICAL][logger]][2012] <class 'syft.core.store.store_memory.MemoryStore'> __delitem__ error <UID: 7429e149c60b4e2bb82cc4644fafa290>.


----relu_ptr----

----res_ptr----

----res_ptr.get()----
tensor([[0.4721, 0.4538, 0.9168, 0.7360]])

----relu_ptr.get()----
ReLU(inplace=True)



In [5]:
# Sequential
seq = th.nn.Sequential()
seq.add_module("fc1", th.nn.Linear(4,2))
seq.add_module("fc2", th.nn.Linear(2,1))

# send
seq_ptr = seq.send(alice_client)
print(f"----seq_ptr----\n{seq_ptr}\n")

# remote call
res_ptr = seq_ptr(th.rand([1,4]))
print(f"----res_ptr----\n{res_ptr}\n")
print(f"----res_ptr.get()----\n{res_ptr.get()}\n")

# remote update state dict
sd2 = OrderedDict(
    th.nn.Sequential(
        collections.OrderedDict([
            ("fc1", th.nn.Linear(4,2)),
            ("fc2", th.nn.Linear(2,1))
        ])
    ).state_dict()
)
sd2_ptr = sd2.send(alice_client)
seq_ptr.load_state_dict(sd2_ptr)

# get
print(f"----seq_ptr.get().state_dict()----\n{seq_ptr.get().state_dict()}\n")
print(f"----sd2----\n{sd2}\n")

[2021-03-25T12:32:07.247627+0800][CRITICAL][logger]][2012] <class 'syft.core.store.store_memory.MemoryStore'> __delitem__ error <UID: 1fe0147a02d24e42813e80025dcbb63b>.


----seq_ptr----

----res_ptr----

----res_ptr.get()----
tensor([[-0.3808]], requires_grad=True)

----seq_ptr.get().state_dict()----
OrderedDict([('fc1.weight', tensor([[ 0.3297, -0.4574,  0.2042, -0.1967],
        [ 0.1805, -0.1045, -0.4810, -0.2268]])), ('fc1.bias', tensor([ 0.2125, -0.1886])), ('fc2.weight', tensor([[ 0.2118, -0.0745]])), ('fc2.bias', tensor([-0.3214]))])

----sd2----
OrderedDict([('fc1.weight', tensor([[ 0.3297, -0.4574,  0.2042, -0.1967],
        [ 0.1805, -0.1045, -0.4810, -0.2268]])), ('fc1.bias', tensor([ 0.2125, -0.1886])), ('fc2.weight', tensor([[ 0.2118, -0.0745]])), ('fc2.bias', tensor([-0.3214]))])



In [6]:
# user defined model
class M(th.nn.Module):
    def __init__(self):
        super(M, self).__init__()
        self.fc1 = th.nn.Linear(4,2)
        self.fc2 = th.nn.Linear(2,1)
        
    def forward(self,x):
        x = self.fc1(x)
        x = self.fc2(x)
        return x
        
m = M()

# send
m_ptr = m.send(alice_client)
print(f"----m_ptr----\n{m_ptr}\n")

# TODO: remote call

# remote update state dict
sd2 = OrderedDict(M().state_dict())
sd2_ptr = sd2.send(alice_client)
m_ptr.load_state_dict(sd2_ptr)

# get
print(f"----m_ptr.get().state_dict()----\n{m_ptr.get().state_dict()}\n")
print(f"----sd2----\n{sd2}\n")

----m_ptr----

----m_ptr.get().state_dict()----
OrderedDict([('fc1.weight', tensor([[-0.2391, -0.1312, -0.1402,  0.1597],
        [-0.4168,  0.0742, -0.0980, -0.2954]])), ('fc1.bias', tensor([ 0.4642, -0.0963])), ('fc2.weight', tensor([[-0.2088, -0.0624]])), ('fc2.bias', tensor([-0.4275]))])

----sd2----
OrderedDict([('fc1.weight', tensor([[-0.2391, -0.1312, -0.1402,  0.1597],
        [-0.4168,  0.0742, -0.0980, -0.2954]])), ('fc1.bias', tensor([ 0.4642, -0.0963])), ('fc2.weight', tensor([[-0.2088, -0.0624]])), ('fc2.bias', tensor([-0.4275]))])

